In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.mit_bih import MITBIH
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss, ClassificationKNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

In [2]:
k = 32
tree_depth = 8
batch_size = 512
device = 'cuda'
train_data_path = r'<>/mitbih_train.csv'  # replace <> with the correct path of the dataset
test_data_path = r'<>/mitbih_test.csv'  # replace <> with the correct path of the dataset

In [3]:
train_data_iter = torch.utils.data.DataLoader(MITBIH(train_data_path),
                                             batch_size=batch_size,
                                             shuffle=True,
                                             num_workers=1,
                                             pin_memory=True,
                                             drop_last=True)

test_data_iter = torch.utils.data.DataLoader(MITBIH(test_data_path),
                                             batch_size=batch_size,
                                             shuffle=True,
                                             num_workers=1,
                                             pin_memory=True)

In [4]:
class ConvBlock(nn.Module):
    def __init__(self):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv1d(32, 32, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv1d(32, 32, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=5, stride=2)
        
    def forward(self, x):
        y = x
        y = self.conv1(y)
        y = self.relu1(y)
        y = self.conv2(y)
        y = y + x
        y = self.relu2(y)
        y = self.pool(y)
        return y


class ECGModel(nn.Module):
    def __init__(self):
        super(ECGModel, self).__init__()
        self.conv = nn.Conv1d(1, 32, kernel_size=5, stride=1, padding=1)
        self.block1 = ConvBlock()
        self.block2 = ConvBlock()
        self.block3 = ConvBlock()
        self.block4 = ConvBlock()
        self.block5 = ConvBlock()
        self.fc1 = nn.Linear(64, 32)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(32, 5)

    def forward(self, x, return_interm=False):
        x = self.conv(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        interm = x.flatten(1)
        x = self.fc1(interm)
        x = self.relu(x)
        x = self.fc2(x)
        
        if return_interm:
            return x, interm
        
        return x

In [5]:
knn_crt = ClassificationKNNLoss(k=k).to(device)

def train(model, loader, optimizer, device):
    model.train()

    total_loss = 0
    for iteration, (batch, target) in enumerate(loader):
        batch = batch.to(device)
        target = target.to(device)
        outputs, interm = model(batch, return_interm=True)
        mse_loss = F.cross_entropy(outputs, target)
        mse_loss = mse_loss.sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(interm, target)
            if torch.isinf(knn_loss):
                knn_loss = torch.tensor(0).to(device)
        except ValueError:
            knn_loss = torch.tensor(0).to(device)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(loader)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | CLS Loss: {mse_loss.item()}")

    return total_loss / len(loader)

@torch.no_grad()
def test(model, loader, device):
    model.eval()
    
    correct = 0
    for iteration, (batch, target) in enumerate(loader):
        batch = batch.to(device)
        target = target.to(device)
        y_pred = model(batch).argmax(dim=-1)
        correct += y_pred.eq(target.view(-1).data).sum()
    
    return correct / len(loader.dataset)

In [6]:
epochs = 200
lr = 1e-3
log_every = 10

model = ECGModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
num_params = sum([p.numel() for p in model.parameters()])
print(f'#Params: {num_params}')

#Params: 53957


In [7]:
best_valid_acc = 0
losses = []
train_accs = []
val_accs = []
for epoch in range(1, epochs + 1):
    loss = train(model, train_data_iter, optimizer, device)
#     print(f"Loss: {loss} =============================")
    losses.append(loss)
    train_acc = test(model, train_data_iter, device)
    train_accs.append(train_acc)
    valid_acc = test(model, test_data_iter, device)
    val_accs.append(valid_acc)
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
    if epoch % 1 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
              f'Train: {train_acc:.4f}, Valid: {valid_acc:.4f}, '
              f'Best: {best_valid_acc:.4f}')

Epoch 1 / 200 | iteration 0 / 171 | Total Loss: 7.338754177093506 | KNN Loss: 5.834799289703369 | CLS Loss: 1.5039548873901367
Epoch 1 / 200 | iteration 10 / 171 | Total Loss: 5.342403888702393 | KNN Loss: 4.256844997406006 | CLS Loss: 1.0855587720870972
Epoch 1 / 200 | iteration 20 / 171 | Total Loss: 4.604902744293213 | KNN Loss: 3.9738428592681885 | CLS Loss: 0.6310598850250244
Epoch 1 / 200 | iteration 30 / 171 | Total Loss: 4.551351070404053 | KNN Loss: 3.9501264095306396 | CLS Loss: 0.6012245416641235
Epoch 1 / 200 | iteration 40 / 171 | Total Loss: 4.384238243103027 | KNN Loss: 3.87967848777771 | CLS Loss: 0.5045595765113831
Epoch 1 / 200 | iteration 50 / 171 | Total Loss: 4.387600898742676 | KNN Loss: 3.8757076263427734 | CLS Loss: 0.5118935108184814
Epoch 1 / 200 | iteration 60 / 171 | Total Loss: 4.261226177215576 | KNN Loss: 3.863406181335449 | CLS Loss: 0.3978199064731598
Epoch 1 / 200 | iteration 70 / 171 | Total Loss: 4.3324408531188965 | KNN Loss: 3.8777098655700684 | CL

Epoch 4 / 200 | iteration 80 / 171 | Total Loss: 3.843834638595581 | KNN Loss: 3.724855661392212 | CLS Loss: 0.1189790666103363
Epoch 4 / 200 | iteration 90 / 171 | Total Loss: 3.889632225036621 | KNN Loss: 3.6959853172302246 | CLS Loss: 0.19364680349826813
Epoch 4 / 200 | iteration 100 / 171 | Total Loss: 3.847315549850464 | KNN Loss: 3.7167088985443115 | CLS Loss: 0.13060657680034637
Epoch 4 / 200 | iteration 110 / 171 | Total Loss: 3.943624496459961 | KNN Loss: 3.7300667762756348 | CLS Loss: 0.21355783939361572
Epoch 4 / 200 | iteration 120 / 171 | Total Loss: 3.8840489387512207 | KNN Loss: 3.7766077518463135 | CLS Loss: 0.1074412390589714
Epoch 4 / 200 | iteration 130 / 171 | Total Loss: 3.8714261054992676 | KNN Loss: 3.7358646392822266 | CLS Loss: 0.1355615109205246
Epoch 4 / 200 | iteration 140 / 171 | Total Loss: 3.9323315620422363 | KNN Loss: 3.7160022258758545 | CLS Loss: 0.21632935106754303
Epoch 4 / 200 | iteration 150 / 171 | Total Loss: 3.868086338043213 | KNN Loss: 3.7704

Epoch 7 / 200 | iteration 160 / 171 | Total Loss: 3.749708414077759 | KNN Loss: 3.7052414417266846 | CLS Loss: 0.04446694999933243
Epoch 7 / 200 | iteration 170 / 171 | Total Loss: 3.840590476989746 | KNN Loss: 3.7238759994506836 | CLS Loss: 0.11671452969312668
Epoch: 007, Loss: 3.8055, Train: 0.9751, Valid: 0.9724, Best: 0.9737
Epoch 8 / 200 | iteration 0 / 171 | Total Loss: 3.7702958583831787 | KNN Loss: 3.7018682956695557 | CLS Loss: 0.0684276595711708
Epoch 8 / 200 | iteration 10 / 171 | Total Loss: 3.813961982727051 | KNN Loss: 3.7236132621765137 | CLS Loss: 0.09034878015518188
Epoch 8 / 200 | iteration 20 / 171 | Total Loss: 3.771840810775757 | KNN Loss: 3.6930980682373047 | CLS Loss: 0.07874263823032379
Epoch 8 / 200 | iteration 30 / 171 | Total Loss: 3.795667886734009 | KNN Loss: 3.6814167499542236 | CLS Loss: 0.11425109207630157
Epoch 8 / 200 | iteration 40 / 171 | Total Loss: 3.7971737384796143 | KNN Loss: 3.695945978164673 | CLS Loss: 0.10122781991958618
Epoch 8 / 200 | iter

Epoch 11 / 200 | iteration 50 / 171 | Total Loss: 3.7585644721984863 | KNN Loss: 3.6319265365600586 | CLS Loss: 0.1266380250453949
Epoch 11 / 200 | iteration 60 / 171 | Total Loss: 3.759752035140991 | KNN Loss: 3.68269419670105 | CLS Loss: 0.077057845890522
Epoch 11 / 200 | iteration 70 / 171 | Total Loss: 3.824002265930176 | KNN Loss: 3.7147364616394043 | CLS Loss: 0.1092657595872879
Epoch 11 / 200 | iteration 80 / 171 | Total Loss: 3.772289276123047 | KNN Loss: 3.680137872695923 | CLS Loss: 0.09215141087770462
Epoch 11 / 200 | iteration 90 / 171 | Total Loss: 3.7551050186157227 | KNN Loss: 3.672175168991089 | CLS Loss: 0.08292978256940842
Epoch 11 / 200 | iteration 100 / 171 | Total Loss: 3.7618613243103027 | KNN Loss: 3.681504249572754 | CLS Loss: 0.08035717159509659
Epoch 11 / 200 | iteration 110 / 171 | Total Loss: 3.759122848510742 | KNN Loss: 3.623027801513672 | CLS Loss: 0.13609500229358673
Epoch 11 / 200 | iteration 120 / 171 | Total Loss: 3.7889795303344727 | KNN Loss: 3.6959

Epoch 14 / 200 | iteration 120 / 171 | Total Loss: 3.753809690475464 | KNN Loss: 3.6689274311065674 | CLS Loss: 0.0848822072148323
Epoch 14 / 200 | iteration 130 / 171 | Total Loss: 3.7419090270996094 | KNN Loss: 3.6707661151885986 | CLS Loss: 0.07114293426275253
Epoch 14 / 200 | iteration 140 / 171 | Total Loss: 3.7657008171081543 | KNN Loss: 3.708181142807007 | CLS Loss: 0.05751967802643776
Epoch 14 / 200 | iteration 150 / 171 | Total Loss: 3.7609260082244873 | KNN Loss: 3.6992526054382324 | CLS Loss: 0.061673350632190704
Epoch 14 / 200 | iteration 160 / 171 | Total Loss: 3.7297489643096924 | KNN Loss: 3.6697888374328613 | CLS Loss: 0.059960171580314636
Epoch 14 / 200 | iteration 170 / 171 | Total Loss: 3.776679277420044 | KNN Loss: 3.7186031341552734 | CLS Loss: 0.058076124638319016
Epoch: 014, Loss: 3.7413, Train: 0.9839, Valid: 0.9805, Best: 0.9806
Epoch 15 / 200 | iteration 0 / 171 | Total Loss: 3.7490642070770264 | KNN Loss: 3.6808879375457764 | CLS Loss: 0.06817632168531418
Epo

...

Epoch: 185, Loss: 3.6264, Train: 0.9976, Valid: 0.9869, Best: 0.9876
Epoch 186 / 200 | iteration 0 / 171 | Total Loss: 3.5996034145355225 | KNN Loss: 3.5942909717559814 | CLS Loss: 0.0053123896941542625
Epoch 186 / 200 | iteration 10 / 171 | Total Loss: 3.618114471435547 | KNN Loss: 3.615853786468506 | CLS Loss: 0.0022606896236538887
Epoch 186 / 200 | iteration 20 / 171 | Total Loss: 3.621063470840454 | KNN Loss: 3.613410234451294 | CLS Loss: 0.007653196342289448
Epoch 186 / 200 | iteration 30 / 171 | Total Loss: 3.5906076431274414 | KNN Loss: 3.5887577533721924 | CLS Loss: 0.0018499090801924467
Epoch 186 / 200 | iteration 40 / 171 | Total Loss: 3.614187002182007 | KNN Loss: 3.6103265285491943 | CLS Loss: 0.0038604652509093285
Epoch 186 / 200 | iteration 50 / 171 | Total Loss: 3.6237847805023193 | KNN Loss: 3.610759973526001 | CLS Loss: 0.013024895451962948
Epoch 186 / 200 | iteration 60 / 171 | Total Loss: 3.643181324005127 | KNN Loss: 3.6329989433288574 | CLS Loss: 0.0101824980229139

Epoch 189 / 200 | iteration 60 / 171 | Total Loss: 3.592528820037842 | KNN Loss: 3.585294485092163 | CLS Loss: 0.007234451826661825
Epoch 189 / 200 | iteration 70 / 171 | Total Loss: 3.591280937194824 | KNN Loss: 3.5898499488830566 | CLS Loss: 0.0014308879617601633
Epoch 189 / 200 | iteration 80 / 171 | Total Loss: 3.579073429107666 | KNN Loss: 3.5757646560668945 | CLS Loss: 0.0033087963238358498
Epoch 189 / 200 | iteration 90 / 171 | Total Loss: 3.6223948001861572 | KNN Loss: 3.6163384914398193 | CLS Loss: 0.0060562132857739925
Epoch 189 / 200 | iteration 100 / 171 | Total Loss: 3.6112325191497803 | KNN Loss: 3.6046500205993652 | CLS Loss: 0.006582415662705898
Epoch 189 / 200 | iteration 110 / 171 | Total Loss: 3.6086058616638184 | KNN Loss: 3.594255208969116 | CLS Loss: 0.014350628480315208
Epoch 189 / 200 | iteration 120 / 171 | Total Loss: 3.5999295711517334 | KNN Loss: 3.596799612045288 | CLS Loss: 0.003129939315840602
Epoch 189 / 200 | iteration 130 / 171 | Total Loss: 3.59919261

Epoch 192 / 200 | iteration 130 / 171 | Total Loss: 3.637855052947998 | KNN Loss: 3.6268444061279297 | CLS Loss: 0.01101070735603571
Epoch 192 / 200 | iteration 140 / 171 | Total Loss: 3.6233179569244385 | KNN Loss: 3.614175319671631 | CLS Loss: 0.009142590686678886
Epoch 192 / 200 | iteration 150 / 171 | Total Loss: 3.5889978408813477 | KNN Loss: 3.583848714828491 | CLS Loss: 0.005149089731276035
Epoch 192 / 200 | iteration 160 / 171 | Total Loss: 3.6393251419067383 | KNN Loss: 3.6283762454986572 | CLS Loss: 0.010948811657726765
Epoch 192 / 200 | iteration 170 / 171 | Total Loss: 3.6060292720794678 | KNN Loss: 3.5987157821655273 | CLS Loss: 0.007313478272408247
Epoch: 192, Loss: 3.6251, Train: 0.9975, Valid: 0.9870, Best: 0.9876
Epoch 193 / 200 | iteration 0 / 171 | Total Loss: 3.5758273601531982 | KNN Loss: 3.570014476776123 | CLS Loss: 0.005812915973365307
Epoch 193 / 200 | iteration 10 / 171 | Total Loss: 3.5749545097351074 | KNN Loss: 3.5733964443206787 | CLS Loss: 0.0015581520274

Epoch 196 / 200 | iteration 10 / 171 | Total Loss: 3.6029610633850098 | KNN Loss: 3.580819606781006 | CLS Loss: 0.022141383960843086
Epoch 196 / 200 | iteration 20 / 171 | Total Loss: 3.607645273208618 | KNN Loss: 3.6037845611572266 | CLS Loss: 0.003860811935737729
Epoch 196 / 200 | iteration 30 / 171 | Total Loss: 3.6153924465179443 | KNN Loss: 3.6134893894195557 | CLS Loss: 0.0019030848052352667
Epoch 196 / 200 | iteration 40 / 171 | Total Loss: 3.616936445236206 | KNN Loss: 3.612663507461548 | CLS Loss: 0.004272931255400181
Epoch 196 / 200 | iteration 50 / 171 | Total Loss: 3.619908094406128 | KNN Loss: 3.6017515659332275 | CLS Loss: 0.01815658062696457
Epoch 196 / 200 | iteration 60 / 171 | Total Loss: 3.6128487586975098 | KNN Loss: 3.608612537384033 | CLS Loss: 0.004236267879605293
Epoch 196 / 200 | iteration 70 / 171 | Total Loss: 3.5964102745056152 | KNN Loss: 3.593170166015625 | CLS Loss: 0.0032401911448687315
Epoch 196 / 200 | iteration 80 / 171 | Total Loss: 3.668031930923462

Epoch 199 / 200 | iteration 70 / 171 | Total Loss: 3.5943009853363037 | KNN Loss: 3.5925846099853516 | CLS Loss: 0.0017163126030936837
Epoch 199 / 200 | iteration 80 / 171 | Total Loss: 3.597454786300659 | KNN Loss: 3.5926666259765625 | CLS Loss: 0.004788219463080168
Epoch 199 / 200 | iteration 90 / 171 | Total Loss: 3.6179463863372803 | KNN Loss: 3.605574131011963 | CLS Loss: 0.012372189201414585
Epoch 199 / 200 | iteration 100 / 171 | Total Loss: 3.617128372192383 | KNN Loss: 3.605790376663208 | CLS Loss: 0.011337882839143276
Epoch 199 / 200 | iteration 110 / 171 | Total Loss: 3.601569175720215 | KNN Loss: 3.578174114227295 | CLS Loss: 0.023395152762532234
Epoch 199 / 200 | iteration 120 / 171 | Total Loss: 3.665844440460205 | KNN Loss: 3.6365015506744385 | CLS Loss: 0.029342852532863617
Epoch 199 / 200 | iteration 130 / 171 | Total Loss: 3.6328747272491455 | KNN Loss: 3.616969108581543 | CLS Loss: 0.01590556465089321
Epoch 199 / 200 | iteration 140 / 171 | Total Loss: 3.626754760742

In [8]:
test(model, test_data_iter, device)

tensor(0.9878, device='cuda:0')

In [9]:
plt.figure()
plt.plot(losses, label='train loss')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
plt.figure()
plt.plot(train_accs, label='train accuracy')
plt.plot(val_accs, label='validation accuracy')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
test_samples = torch.tensor([])
projections = torch.tensor([])
labels = torch.tensor([])

with torch.no_grad():
    for x, y in tqdm(test_data_iter):
        test_samples = torch.cat([test_samples, x])
        labels = torch.cat([labels, y])
        x = x.to(device)
        _, interm = model(x, True)
        projections = torch.cat([projections, interm.detach().cpu().flatten(1)])

  0%|          | 0/43 [00:00<?, ?it/s]

In [12]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
clusters = DBSCAN(eps=2, min_samples=10).fit_predict(projections)

In [14]:
print(f"Number of inliers: {sum(clusters != -1) / len(clusters)}")

Number of inliers: 0.9780731807592161


In [15]:
perplexity = 100
p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [16]:
tree_dataset = list(zip(test_samples.flatten(1)[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [17]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [18]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [19]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 400
log_interval = 100
use_cuda = device != 'cpu'

In [20]:
tree = SDT(input_dim=test_samples.shape[2], output_dim=len(set(clusters)) - 1, depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [21]:
losses = []
accs = []
sparsity = []

In [22]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

Average sparseness: 0.0
layer 0: 0.0
layer 1: 0.0
layer 2: 0.0
layer 3: 0.0
layer 4: 0.0
layer 5: 0.0
layer 6: 0.0
Epoch: 00 | Batch: 000 / 042 | Total loss: 1.413 | Reg loss: 0.009 | Tree loss: 1.413 | Accuracy: 0.064453 | 0.308 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 01 | Batch: 000 / 042 | Total loss: 1.185 | Reg loss: 0.004 | Tree loss: 1.185 | Accuracy: 0.794922 | 0.22 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 02 | Batch: 000 / 042 | Total loss: 1.109 | Reg loss: 0.007 | Tree loss: 1.109 | Accuracy: 0.744141 | 0.218 sec/iter
Average sparseness: 0.9840425531914894
layer

Epoch: 23 | Batch: 000 / 042 | Total loss: 0.677 | Reg loss: 0.021 | Tree loss: 0.677 | Accuracy: 0.763672 | 0.227 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 24 | Batch: 000 / 042 | Total loss: 0.688 | Reg loss: 0.021 | Tree loss: 0.688 | Accuracy: 0.759766 | 0.227 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 25 | Batch: 000 / 042 | Total loss: 0.615 | Reg loss: 0.022 | Tree loss: 0.615 | Accuracy: 0.808594 | 0.228 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4

Epoch: 46 | Batch: 000 / 042 | Total loss: 0.650 | Reg loss: 0.024 | Tree loss: 0.650 | Accuracy: 0.783203 | 0.229 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 47 | Batch: 000 / 042 | Total loss: 0.607 | Reg loss: 0.024 | Tree loss: 0.607 | Accuracy: 0.796875 | 0.229 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 48 | Batch: 000 / 042 | Total loss: 0.627 | Reg loss: 0.024 | Tree loss: 0.627 | Accuracy: 0.802734 | 0.229 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4

Epoch: 69 | Batch: 000 / 042 | Total loss: 0.615 | Reg loss: 0.025 | Tree loss: 0.615 | Accuracy: 0.808594 | 0.229 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 70 | Batch: 000 / 042 | Total loss: 0.624 | Reg loss: 0.025 | Tree loss: 0.624 | Accuracy: 0.804688 | 0.229 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 71 | Batch: 000 / 042 | Total loss: 0.652 | Reg loss: 0.025 | Tree loss: 0.652 | Accuracy: 0.792969 | 0.229 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4

layer 6: 0.9840425531914895
Epoch: 92 | Batch: 000 / 042 | Total loss: 0.685 | Reg loss: 0.025 | Tree loss: 0.685 | Accuracy: 0.765625 | 0.228 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 93 | Batch: 000 / 042 | Total loss: 0.626 | Reg loss: 0.025 | Tree loss: 0.626 | Accuracy: 0.787109 | 0.228 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 94 | Batch: 000 / 042 | Total loss: 0.612 | Reg loss: 0.025 | Tree loss: 0.612 | Accuracy: 0.798828 | 0.228 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3

...

Epoch: 299 | Batch: 000 / 042 | Total loss: 0.524 | Reg loss: 0.025 | Tree loss: 0.524 | Accuracy: 0.841797 | 0.232 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 300 | Batch: 000 / 042 | Total loss: 0.588 | Reg loss: 0.025 | Tree loss: 0.588 | Accuracy: 0.818359 | 0.232 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 301 | Batch: 000 / 042 | Total loss: 0.579 | Reg loss: 0.025 | Tree loss: 0.579 | Accuracy: 0.833984 | 0.232 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
laye

Epoch: 322 | Batch: 000 / 042 | Total loss: 0.621 | Reg loss: 0.025 | Tree loss: 0.621 | Accuracy: 0.785156 | 0.232 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 323 | Batch: 000 / 042 | Total loss: 0.678 | Reg loss: 0.025 | Tree loss: 0.678 | Accuracy: 0.769531 | 0.232 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 324 | Batch: 000 / 042 | Total loss: 0.648 | Reg loss: 0.025 | Tree loss: 0.648 | Accuracy: 0.789062 | 0.232 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
laye

layer 6: 0.9840425531914895
Epoch: 345 | Batch: 000 / 042 | Total loss: 0.617 | Reg loss: 0.025 | Tree loss: 0.617 | Accuracy: 0.796875 | 0.233 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 346 | Batch: 000 / 042 | Total loss: 0.571 | Reg loss: 0.025 | Tree loss: 0.571 | Accuracy: 0.826172 | 0.233 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 347 | Batch: 000 / 042 | Total loss: 0.587 | Reg loss: 0.025 | Tree loss: 0.587 | Accuracy: 0.820312 | 0.233 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
laye

Epoch: 368 | Batch: 000 / 042 | Total loss: 0.615 | Reg loss: 0.025 | Tree loss: 0.615 | Accuracy: 0.791016 | 0.233 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 369 | Batch: 000 / 042 | Total loss: 0.591 | Reg loss: 0.025 | Tree loss: 0.591 | Accuracy: 0.810547 | 0.233 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 370 | Batch: 000 / 042 | Total loss: 0.608 | Reg loss: 0.025 | Tree loss: 0.608 | Accuracy: 0.808594 | 0.233 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
laye

Epoch: 391 | Batch: 000 / 042 | Total loss: 0.606 | Reg loss: 0.025 | Tree loss: 0.606 | Accuracy: 0.804688 | 0.233 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 392 | Batch: 000 / 042 | Total loss: 0.640 | Reg loss: 0.025 | Tree loss: 0.640 | Accuracy: 0.791016 | 0.233 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 393 | Batch: 000 / 042 | Total loss: 0.600 | Reg loss: 0.025 | Tree loss: 0.600 | Accuracy: 0.820312 | 0.233 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
laye

In [23]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
# plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Tree Visualization

In [25]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Average height: 5.666666666666667


# Extract Rules

# Accumulate samples in the leaves

In [26]:
print(f"Number of patterns: {len(root.get_leaves())}")

Number of patterns: 24


In [27]:
method = 'greedy'

In [28]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

/home/eitan.k/.local/lib/python3.6/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


# Tighten boundaries

In [29]:
attr_names = [f"T_{i}" for i in range(test_samples.shape[2])]
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")

138
============== Pattern 1 ==============
507
============== Pattern 2 ==============
============== Pattern 3 ==============
============== Pattern 4 ==============
============== Pattern 5 ==============
============== Pattern 6 ==============
============== Pattern 7 ==============
============== Pattern 8 ==============
485
============== Pattern 9 ==============
============== Pattern 10 ==============
10797
============== Pattern 11 ==============
7571
============== Pattern 12 ==============
10
============== Pattern 13 ==============
============== Pattern 14 ==============
1903
============== Pattern 15 ==============
============== Pattern 16 ==============
============== Pattern 17 ==============
============== Pattern 18 ==============
============== Pattern 19 ==============
============== Pattern 20 ==============
============== Pattern 21 ==============
============== Pattern 22 ==============
============== Pattern 23 ==============
============== Pattern 24 =========

/home/eitan.k/EntangledExplainableClustering/soft_decision_tree/sdt_model.py:14: RuntimeWarning: divide by zero encountered in log
  return np.log(1 / (1 - x))
